## Task 1 - SQL Queries

**Customer Table: "customer"**                                              
                                                                    
| id | first_name | last_name  | gender | date_of_birth | country |        
|----|------------|------------|--------|---------------|---------|     
| 1  | Jason      | Smith      | male   | 1982-05-28    | USA     |     
| 2  | Max        | Mustermann | male   | 1980-07-18    | Germany |     
| 3  | Will       | Myer       | male   | 1981-03-30    | England |     
| 4  | Christin   | Dawn       | female | 1978-08-02    | USA     |     
| 5  | Angela     | Gutierez   | female | 1986-01-16    | Spain   |     
| 6  | Peter      | Jackson    | male   | 1958-12-05    | USA     |     
                                                                        

**Order Table: "order"**

| id | order_nr   | sum    | fk_customer |
|----|------------|--------|-------------|
| 1  | 2783292423 | 100.85 | 2           |
| 2  | 4784232411 | 77.34  | 3           |
| 3  | 3783292423 | 30.99  | 5           |
| 4  | 9368315313 | 33.55  | 2           |

**Order Item Table: "order_item"**

| id | sku     | fk_order |
|----|---------|----------|
| 1  | ABCDEF  | 1        |
| 2  | ABCDEF  | 1        |
| 3  | OHSDLF  | 1        |
| 4  | 1737234 | 2        |
| 5  | KLSHA   | 3        |
| 6  | OHSDLF  | 3        |
| 7  | GHJSK   | 4        |


Given the tables above.

1. Write a query which select all female customers

    **SELECT * FROM customer WHERE gender="female"**

2. Write a query which prints out all customer names with the number of orders they did

    **SELECT customer.first_name, customer.last_name, COUNT(order.id) as 'order_count' FROM customer LEFT JOIN order ON order.fk_customer = customer.id**

3. Write a query which prints out customers with the money they spend excluding customers without any orders

    **SELECT customer.*, SUM(order.sum) as "customer_expense" FROM customer INNER JOIN order ON order.fk_customer = customer.id GROUP BY customer.id**

4. Write a query which prints out the order nr of all orders with at least 2 items

    **SELECT order.order_nr, count(order_item.id) as 'item_count' FROM order INNER JOIN order_item ON order.id = order_item.fk_order GROUP BY order.order_nr HAVING count(order_item.id) >= 2**

---

## Task # 2 Python Code

Create a Python script that fetches tweets with hashtag **#ecommerce** from the previous day.

Sample scenario: August 1 2022 extraction will fetch the data from July 31 2022). The file should be a list of json objects.

(**Optional**: If you can include a feature to store the output file into a cloud storage, please do so.)

## Pre-requisites:

**Packages to install:**
* pip install tweepy
* pip install datetime

**API Tokens**
* Retrieve BEARER_TOKEN from personal twitter developer account
* Execute the following in your terminal:
     export SEARCH_API_BEARER_TOKEN=XXXXBearerTokenHereXXXX

**Project Notes:**
* Project Directory: /task2
* Date Input and Output is in UTC format
* Project Output: /task2/task2-output.json - file name can be changed in the main function, file_name variable
* No cloud upload feature

---

## Task # 3 Table Design
From the data that you got from task#2, imagine reading millions or billions of rows from it. Describe a way on how you will design the table so that processing or querying the table will be optimized.


The json data extracted from Twitter contains a series of nested objects. I will be doing a hortizontal partioning for the table design. 

Partioning can be based on the following:
1. UserId - it will be faster to locate and fetch all Tweets related to the user.
2. TweetId - faster if we will need to query specific Tweets
3. TweetId w/ Creation datetime - for aggregation you wont need to have a separated creation datetime table/index
4. Creation timestamp - for fetching tweets based on creation date


It will also be important to design the table to the correct data types of the Twitter data. For example, using integer types for IDs, and for the column where Twitter text will be stored, a limit can be aligned to the max characters of a tweet instead of having the datatype as varchar max.

For querying, using Views can also be a way to reduce execution time especially those queries with complex aggregation and joins.



## Task # 4 Data Pipeline Design in Cloud Setup
Assuming you want to deploy an automated solution for task#2 in a cloud setup to be available to an end user via a BI platform. Give a short high level description of a possible approach and some considerations that would affect your choices.

For task #2, I will take advantage of Amazon Web Services as the cloud platform.
The raw twitter data can be stored in an S3 bucket as the landing point. Performing data discovery and data integration jobs will be done via AWS Glue. 
I have considered AWS Glue to be used for the raw twitter data since it has the capability to do automatic data discovery, the Data Catalog feature, and automatically creates scripts for ETL.
Considering Twitter's datasets can contain billions of rows, how the raw inputs are structured, and the complexity the queries will be, data processing will be done via AWS Redshift or AWS Athena. From here, insights can be presented to end users via Tableau, AWS QuickSight or any BI platforms that can be integrated with Redshift/Athena.
